In [ ]:
# tickets_pipeline.py

import pandas as pd
import janitor
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

def main():
    #  Load & basic cleaning
    df = (
        pd.read_csv("/content/dataset-tickets-multi-lang-4-20k.csv", encoding="utf-8")
          .clean_names()
          .remove_empty()
          .drop_duplicates()
    )

    #  Drop all tag_* columns except tag_1
    drop_tags = [c for c in df.columns if c.startswith("tag_") and c != "tag_1"]
    df.drop(columns=drop_tags, inplace=True)

    #  Combine text fields
    df["subject"] = df["subject"].fillna("")
    df["body"]    = df["body"].fillna("")
    df["text"]    = (df["subject"] + " " + df["body"]).str.strip()
    df = df[df["text"] != ""]

    #  Fill missing metadata with "Unknown"
    for col in ["queue", "priority", "language", "tag_1"]:
        if col in df.columns:
            df[col] = df[col].fillna("Unknown")

    #  Merge very low‑frequency categories (<5%) into "Other"
    def merge_low_freq(series, thresh=0.05):
        freqs = series.value_counts(normalize=True)
        rare  = freqs[freqs < thresh].index
        return series.replace(rare, "Other")

    for cat in ["type", "queue", "priority", "language", "tag_1"]:
        if cat in df.columns:
            df[cat] = merge_low_freq(df[cat])

    #  Encode target labels → numeric
    y_raw = df["type"]
    le = LabelEncoder()
    y = le.fit_transform(y_raw)

    #  Save mapping for later
    print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

    X = df[["text", "queue", "priority", "language", "tag_1"]]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        stratify=y,
        test_size=0.20,
        random_state=42
    )

    #  Preprocessing: TF-IDF for text + TargetEncoder for metadata
    preprocessor = ColumnTransformer(transformers=[
        ("text_tfidf",
         TfidfVectorizer(strip_accents="unicode", max_features=20_000),
         "text"),
        ("meta_te",
         ce.TargetEncoder(cols=["queue","priority","language","tag_1"]),
         ["queue","priority","language","tag_1"]),
    ], remainder="drop")

    #  Full pipeline: preprocess → SMOTE → XGBoost
    pipe = ImbPipeline(steps=[
        ("preproc", preprocessor),
        ("smote", SMOTE(random_state=42)),
        ("clf", XGBClassifier(
                    eval_metric="mlogloss",
                    use_label_encoder=False,
                    random_state=42
               )),
    ])

    #  Train
    pipe.fit(X_train, y_train)

    import joblib
    joblib.dump(pipe, "ticket_type_model.pkl")
    print(" Model saved as ticket_type_model.pkl")


    y_pred_num = pipe.predict(X_test)
    y_pred     = le.inverse_transform(y_pred_num)
    y_true     = le.inverse_transform(y_test)
    df["text"]    = (df["subject"] + " " + df["body"]).str.strip()

    print("\nClassification report:")
    print(classification_report(y_true, y_pred, digits=4))

if __name__ == "__main__":
    main()


In [ ]:
import os
import time
import uuid
import pandas as pd
from sentence_transformers import SentenceTransformer
from supabase import create_client, Client

SUPABASE_KEY = ""
SUPABASE_URL=""
TABLE_NAME = ""

BATCH_SIZE = 100       # number of rows per insert
BATCH_DELAY = 0.5      # delay between batches (seconds)
CHUNK_MAX_LEN = 200    # max words per chunk

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
embed_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

df = pd.read_csv("/content/dataset-tickets-multi-lang-4-20k.csv")

# Combine subject + body
df["subject"] = df["subject"].fillna("")
df["body"] = df["body"].fillna("")
df["text"] = df["subject"] + " " + df["body"]

# If columns missing → default
if "resolution" not in df.columns:
    df["resolution"] = "No resolution available"
if "type" not in df.columns:
    df["type"] = "Unknown"

print(f" Loaded {len(df)} tickets")

def chunk_text(text, max_len=CHUNK_MAX_LEN):
    words = text.split()
    for i in range(0, len(words), max_len):
        yield " ".join(words[i:i+max_len])

def insert_batch_safe(batch):
    try:
        supabase.table(TABLE_NAME).insert(batch).execute()
        return True
    except Exception as e:
        print("Insert failed, retrying in 2s...", e)
        time.sleep(2)
        try:
            supabase.table(TABLE_NAME).insert(batch).execute()
            print("Retry success!")
            return True
        except Exception as e2:
            print("❌ Retry failed:", e2)
            return False

batch_data = []
total_inserted = 0

for idx, row in df.iterrows():
    ticket_id = str(row.get("ticket_id", uuid.uuid4()))
    full_text = row["text"]
    resolution = row["resolution"]
    ticket_type = row["type"]

    # Split into chunks
    for chunk_idx, chunk in enumerate(chunk_text(full_text)):
        # Embed chunk
        embedding = embed_model.encode(chunk).tolist()

        metadata = {
            "ticket_id": ticket_id,
            "ticket_type": ticket_type,
            "resolution": resolution,
            "chunk_idx": chunk_idx
        }

        record = {
            "chunk_text": chunk,
            "metadata": metadata,
            "embedding": embedding
        }

        batch_data.append(record)

    if len(batch_data) >= BATCH_SIZE:
        print(f" Inserting batch of {len(batch_data)} rows...")
        success = insert_batch_safe(batch_data)
        if success:
            total_inserted += len(batch_data)
        else:
            print(" Batch skipped due to repeated failure.")
        batch_data = []
        time.sleep(BATCH_DELAY)

if batch_data:
    print(f"🚀 Inserting final {len(batch_data)} rows...")
    success = insert_batch_safe(batch_data)
    if success:
        total_inserted += len(batch_data)

print(f"DONE! Total inserted rows: {total_inserted}")


In [ ]:
import joblib
import numpy as np
from supabase import create_client, Client
from sentence_transformers import SentenceTransformer
import pandas as pd

SUPABASE_KEY = ""
SUPABASE_URL=""
# TABLE_NAME = "document_chunks"

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

clf = joblib.load("ticket_type_model.pkl")
embed_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")  # must match DB dim

label_mapping = {
    0: "Change",
    1: "Incident",
    2: "Problem",
    3: "Request"
}

EXPECTED_META = ["queue", "priority", "language", "tag_1"]


def preprocess_ticket(subject: str, body: str, metadata: dict) -> dict:
    """Preprocess a single ticket to match training pipeline"""
    subject = subject if subject else ""
    body = body if body else ""
    text = (subject + " " + body).strip()

    # Fill missing metadata with 'Unknown'
    clean_meta = {}
    for key in EXPECTED_META:
        clean_meta[key] = metadata.get(key, "Unknown") if metadata else "Unknown"

    return {
        "text": text,
        "queue": clean_meta["queue"],
        "priority": clean_meta["priority"],
        "language": clean_meta["language"],
        "tag_1": clean_meta["tag_1"]
    }

def classify_ticket(ticket_features: dict) -> str:
    """
    Predict ticket type using saved classifier.
    Returns decoded label (Change/Incident/Problem/Request).
    """
    # Must pass a DataFrame with correct column names
    X = pd.DataFrame([{
        "text": ticket_features["text"],
        "queue": ticket_features["queue"],
        "priority": ticket_features["priority"],
        "language": ticket_features["language"],
        "tag_1": ticket_features["tag_1"]
    }])

    pred_num = clf.predict(X)[0]  # numeric prediction
    return label_mapping.get(pred_num, "Unknown")

def embed_text(text: str) -> list:
    """Embed text for Supabase RAG search"""
    return embed_model.encode(text).tolist()

def retrieve_similar_chunks(query_text: str, match_count: int = 3):
    """
    Calls Supabase pgvector function `match_chunks()`
    to retrieve top-k similar ticket chunks.
    """
    query_embedding = embed_text(query_text)
    response = supabase.rpc(
        "match_chunks",
        {
            "query_embedding": query_embedding,
            "match_count": match_count
        }
    ).execute()
    return response.data

def save_new_ticket_to_supabase(subject, body, metadata, predicted_type):
    """Embed and insert the new ticket into Supabase for future retrieval."""
    text = (subject + " " + body).strip()
    embedding = embed_model.encode(text).tolist()

    # Combine metadata + predicted type
    full_metadata = {
        "predicted_type": predicted_type,
        **(metadata or {})  # include queue, priority, etc.
    }

    record = {
        # let Supabase auto-generate 'id' if table has default uuid
        "chunk_text": text,
        "metadata": full_metadata,
        "embedding": embedding
    }

    supabase.table("document_chunks").insert(record).execute()
    print(f" New ticket saved in Supabase ")


def classify_retrieve_save(subject: str, body: str, metadata: dict = None, top_k: int = 3):
    """
    Full pipeline:
    1. Preprocess ticket
    2. Predict ticket type (decoded label)
    3. Retrieve top-k similar historical chunks from Supabase
    4. Save new ticket into Supabase for future retrieval
    """

    # Step 1: Preprocess
    ticket_features = preprocess_ticket(subject, body, metadata)
    print(f" Cleaned text: {ticket_features['text'][:100]}...")

    # Step 2: Classify
    predicted_type = classify_ticket(ticket_features)
    print(f"Predicted Ticket Type: {predicted_type}")

    # Step 3: Retrieve similar chunks
    similar_chunks = retrieve_similar_chunks(ticket_features["text"], match_count=top_k)
    print("\n Top Similar Historical Tickets:")
    for match in similar_chunks:
        print(f"ID: {match['id']}")
        print(f"Chunk: {match['chunk_text'][:100]}...")
        print(f"Similarity: {round(match['similarity'], 4)}\n")

    # Step 4: Save new ticket into Supabase
    save_new_ticket_to_supabase(subject, body, metadata, predicted_type)

    return {
        "predicted_type": predicted_type,
        "similar_chunks": similar_chunks
    }

if __name__ == "__main__":
    subject = "Cannot login after OS update"
    body = "After updating Windows, I cannot login to my laptop. It says wrong password even though it's correct."
    metadata = {
        "queue": "Hardware",
        "priority": "High",
        "language": "English",
        "tag_1": "Login Issue"
    }

    result = classify_retrieve_save(subject, body, metadata, top_k=3)

    print("\n FINAL RESULT:")
    print("Predicted Ticket Type:", result["predicted_type"])
    for match in result["similar_chunks"]:
        print(f"- Similar Chunk: {match['chunk_text'][:80]}... (Score: {round(match['similarity'],4)})")